In [1]:
from functools import lru_cache

import duckdb
import requests
import pandas as pd

pd.set_option('display.max_rows', None)

In [3]:
con = duckdb.connect("/data/duckdb/github_dw.duckdb", read_only=True)

In [5]:
ls /home/jovyan/.duckdb/stored_secrets/

s3_secret.duckdb_secret


In [4]:
_ = con.execute(
"""
    INSTALL delta;
    LOAD delta;
    
    INSTALL httpfs;
    LOAD httpfs;
    
    CREATE OR REPLACE PERSISTENT SECRET s3_secret (
    TYPE s3,
    PROVIDER config,
    KEY_ID '',
    SECRET '',
    URL_STYLE 'path',
    USE_SSL false,
    ENDPOINT 's3.ap-southeast-1.amazonaws.com',
    REGION 'ap-southeast-1'
    );
"""
)

In [4]:
_ = con.execute(
"""
    INSTALL delta;
    LOAD delta;
    
    INSTALL httpfs;
    LOAD httpfs;
    
    CREATE OR REPLACE PERSISTENT SECRET docker_secret (
    TYPE s3,
    PROVIDER config,
    KEY_ID 'datalake',
    SECRET 'datalake',
    URL_STYLE 'path',
    USE_SSL false,
    ENDPOINT 'minio:9000',
    REGION 'ap-southeast-1'
    );
"""
)

In [4]:
_ = con.execute(
"""
    INSTALL delta;
    LOAD delta;
    
    INSTALL httpfs;
    LOAD httpfs;
    
    CREATE OR REPLACE PERSISTENT SECRET local_secret (
    TYPE s3,
    PROVIDER config,
    KEY_ID 'datalake',
    SECRET 'datalake',
    URL_STYLE 'path',
    USE_SSL false,
    ENDPOINT 'localhost:9050',
    REGION 'ap-southeast-1'
    );
"""
)

In [5]:
_ = con.execute(
"""
    DROP SECRET docker_secret;
"""
)

InvalidInputException: Invalid Input Error: Failed to remove non-existent secret with name 'docker_secret'

In [7]:
df = con.sql("FROM duckdb_secrets()")
df

┌───────────────┬─────────┬──────────┬────────────┬────────────┬─────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│     name      │  type   │ provider │ persistent │  storage   │          scope          │                                                                                             secret_string                                                                                              │
│    varchar    │ varchar │ varchar  │  boolean   │  varchar   │        varchar[]        │                                                                                                varchar                                                                                                 │
├───────────────┼─────────┼──────────┼────────────┼────────────┼─────────────────────────┼──────────────────────────────────

In [80]:
_ = con.execute("USE raw")

In [9]:
con.sql("FROM which_secret('s3://covid-data-pipeline/covid19/covid19datahub', 's3')")

┌───────────────┬────────────┬────────────┐
│     name      │ persistent │  storage   │
│    varchar    │  varchar   │  varchar   │
├───────────────┼────────────┼────────────┤
│ docker_secret │ PERSISTENT │ local_file │
└───────────────┴────────────┴────────────┘

In [4]:
con.sql("""
SELECT COUNT(1), location_id
FROM curated.dim_location
GROUP BY location_id
ORDER BY 1 DESC
""")
# 13684

┌──────────┬──────────────────────────────────┐
│ count(1) │           location_id            │
│  int64   │             varchar              │
├──────────┼──────────────────────────────────┤
│        1 │ 4f7efc023ded10e1b7d65dbd271a8755 │
│        1 │ 32a7a102817188bc1ae158f08a052be0 │
│        1 │ bdb13f793a19fbcd51f3da2b1e5a1372 │
│        1 │ 9586b7c7e9eed3cfeed8d964f8f20e9c │
│        1 │ 2e627578dde8df77db90a769336a74ed │
│        1 │ 36b3d13570908f07b14398cf79f53aed │
│        1 │ 9dea06205b1d273b3dc68832267a4246 │
│        1 │ 2fce982804dca775b165febedfd5ba28 │
│        1 │ 85031bba487c8254ce0555c36fec1d92 │
│        1 │ c038a29dd552432c8ec1e365f0675239 │
│        · │                ·                 │
│        · │                ·                 │
│        · │                ·                 │
│        1 │ ebe73fd45a0fef3e620a65c78b48a1c5 │
│        1 │ fadfe410f7aefe9b671bec70535ab972 │
│        1 │ 493df94d610d7c7a1e7c98114ebc394a │
│        1 │ bfb1f797af74126918f9fb375bf

In [16]:
location_df = con.sql("""
    SELECT COUNT(id),
        COUNT(CASE WHEN combined_key IS NULL THEN 1 ELSE NULL END) AS combined_key_cnt_null , 
        COUNT(CASE WHEN province_state IS NULL THEN 1 ELSE NULL END) AS province_state_cnt_null, 
        COUNT(CASE WHEN country_region IS NULL THEN 1 ELSE NULL END) AS country_region_cnt_null, 
        COUNT(CASE WHEN latitude IS NULL THEN 1 ELSE NULL END) AS latitude_cnt_null, 
        COUNT(CASE WHEN longitude IS NULL THEN 1 ELSE NULL END) AS longitude_cnt_null, 
        COUNT(CASE WHEN fips IS NULL THEN 1 ELSE NULL END) AS fips_cnt_null, 
        COUNT(CASE WHEN admin2 IS NULL THEN 1 ELSE NULL END) AS admin2_cnt_null
    FROM cleansed_location LIMIT 10
""")
location_df

┌───────────┬───────────────────────┬─────────────────────────┬─────────────────────────┬───────────────────┬────────────────────┬───────────────┬─────────────────┐
│ count(id) │ combined_key_cnt_null │ province_state_cnt_null │ country_region_cnt_null │ latitude_cnt_null │ longitude_cnt_null │ fips_cnt_null │ admin2_cnt_null │
│   int64   │         int64         │          int64          │          int64          │       int64       │       int64        │     int64     │      int64      │
├───────────┼───────────────────────┼─────────────────────────┼─────────────────────────┼───────────────────┼────────────────────┼───────────────┼─────────────────┤
│      5322 │                     0 │                       0 │                       0 │                 0 │                  0 │             0 │               0 │
└───────────┴───────────────────────┴─────────────────────────┴─────────────────────────┴───────────────────┴────────────────────┴───────────────┴─────────────────┘

In [17]:
con.sql("""
SELECT COUNT(DISTINCT CASE WHEN administrative_area_level = 1 THEN 1 ELSE NULL END) AS level1,
       COUNT(DISTINCT CASE WHEN administrative_area_level = 2 THEN 1 ELSE NULL END) AS level2,
       COUNT(DISTINCT CASE WHEN administrative_area_level = 3 THEN 1 ELSE NULL END) AS level3,
       country_region
FROM cleansed.cleansed_location 

GROUP BY country_region
HAVING level3 > 0""").to_df()

,level1,level2,level3,country_region
0,0,1,1,US


In [19]:
con.sql("SELECT MIN(last_update), MAX(last_update) FROM covid19_raw.raw_github_csse_daily")

┌──────────────────┬──────────────────┐
│ min(last_update) │ max(last_update) │
│     varchar      │     varchar      │
├──────────────────┼──────────────────┤
│ 1/22/2020 17:00  │ 4/6/20 9:37      │
└──────────────────┴──────────────────┘

In [17]:
con.sql("SELECT * FROM cleansed_location LIMIT 10")

┌──────────────────────────────────┬──────────────────────────────┬─────────────┬──────────────┬────────────────┬────────────────┬───────┬──────────────┐
│                id                │         combined_key         │  latitude   │  longitude   │ province_state │ country_region │ fips  │    admin2    │
│             varchar              │           varchar            │   double    │    double    │    varchar     │    varchar     │ int32 │   varchar    │
├──────────────────────────────────┼──────────────────────────────┼─────────────┼──────────────┼────────────────┼────────────────┼───────┼──────────────┤
│ 3650296a82594e3c2c3a77bf8fee7570 │ Sichuan, China               │     30.6171 │     102.7103 │ Sichuan        │ China          │ -9999 │ Unassigned   │
│ 272c321f2dac912a6dbe1d47234120fd │ Sonora, Mexico               │     29.2972 │    -110.3309 │ Sonora         │ Mexico         │ -9999 │ Unassigned   │
│ 8c0b1ac6cc1acc0bb413aed5d724394d │ Thailand                     │   15.870

In [17]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT len, COUNT(1)
    FROM t1
    GROUP BY len
    ORDER BY len DESC
    ;
""")

┌───────┬──────────┐
│  len  │ count(1) │
│ int64 │  int64   │
├───────┼──────────┤
│    19 │  4119846 │
│    16 │       37 │
│    15 │      178 │
│    14 │       73 │
│    13 │    14494 │
│    12 │     8201 │
│    11 │       68 │
└───────┴──────────┘

In [20]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT *
    FROM t1
    WHERE len_last_update = 19
    LIMIT 10;

    -- Example Value: 2023-01-06 04:21:02
""")

┌─────────────────┬──────────────────────────────────┬───────────┬──────────┬───────────┬────────┬────────────────────┬────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬───────┬───────┬───────┐
│ len_last_update │                id                │ confirmed │  deaths  │ recovered │ active │   incident_rate    │ incidence_rate │ case_fatality_ratio │     last_update     │      load_date      │ year  │ month │  day  │
│      int64      │             varchar              │  double   │  double  │  double   │ int64  │       double       │     double     │       double        │       varchar       │      timestamp      │ int64 │ int64 │ int64 │
├─────────────────┼──────────────────────────────────┼───────────┼──────────┼───────────┼────────┼────────────────────┼────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼───────┼───────┼───────┤
│              19 │ 6c576b5de057f82f2e0d9fc2d19ff45d │ 9963697.0 │ 130171.0 │      NULL │   

In [21]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT *
    FROM t1
    WHERE len_last_update = 16
    LIMIT 10;

    -- Example Value: 2021-01-15 17:22
""")

┌─────────────────┬──────────────────────────────────┬───────────┬─────────┬───────────┬────────┬────────────────────┬────────────────┬─────────────────────┬──────────────────┬─────────────────────┬───────┬───────┬───────┐
│ len_last_update │                id                │ confirmed │ deaths  │ recovered │ active │   incident_rate    │ incidence_rate │ case_fatality_ratio │   last_update    │      load_date      │ year  │ month │  day  │
│      int64      │             varchar              │  double   │ double  │  double   │ int64  │       double       │     double     │       double        │     varchar      │      timestamp      │ int64 │ int64 │ int64 │
├─────────────────┼──────────────────────────────────┼───────────┼─────────┼───────────┼────────┼────────────────────┼────────────────┼─────────────────────┼──────────────────┼─────────────────────┼───────┼───────┼───────┤
│              16 │ 039b389ba69595065fb4c9e39bbc5fee │  885616.0 │  7138.0 │  876140.0 │   2338 │ 1642.96893

In [22]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT *
    FROM t1
    WHERE len_last_update = 15
    LIMIT 10;

    -- Example Value: 1/22/2020 17:00
""")

┌─────────────────┬──────────────────────────────────┬───────────┬────────┬───────────┬────────┬───────────────┬────────────────┬─────────────────────┬─────────────────┬─────────────────────┬───────┬───────┬───────┐
│ len_last_update │                id                │ confirmed │ deaths │ recovered │ active │ incident_rate │ incidence_rate │ case_fatality_ratio │   last_update   │      load_date      │ year  │ month │  day  │
│      int64      │             varchar              │  double   │ double │  double   │ int64  │    double     │     double     │       double        │     varchar     │      timestamp      │ int64 │ int64 │ int64 │
├─────────────────┼──────────────────────────────────┼───────────┼────────┼───────────┼────────┼───────────────┼────────────────┼─────────────────────┼─────────────────┼─────────────────────┼───────┼───────┼───────┤
│              15 │ 755f36722d7e163e6ce4438aec2f95d9 │       1.0 │   NULL │      NULL │   NULL │          NULL │           NULL │       

In [23]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT *
    FROM t1
    WHERE len_last_update = 14
    LIMIT 10;

    -- Example Value: 2/1/2020 10:33
""")

┌─────────────────┬──────────────────────────────────┬───────────┬────────┬───────────┬────────┬───────────────┬────────────────┬─────────────────────┬────────────────┬─────────────────────┬───────┬───────┬───────┐
│ len_last_update │                id                │ confirmed │ deaths │ recovered │ active │ incident_rate │ incidence_rate │ case_fatality_ratio │  last_update   │      load_date      │ year  │ month │  day  │
│      int64      │             varchar              │  double   │ double │  double   │ int64  │    double     │     double     │       double        │    varchar     │      timestamp      │ int64 │ int64 │ int64 │
├─────────────────┼──────────────────────────────────┼───────────┼────────┼───────────┼────────┼───────────────┼────────────────┼─────────────────────┼────────────────┼─────────────────────┼───────┼───────┼───────┤
│              14 │ ddf6ce64bc193e0c4914afc6785948d0 │      80.0 │    2.0 │       2.0 │   NULL │          NULL │           NULL │           

In [24]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT *
    FROM t1
    WHERE len_last_update = 13
    LIMIT 10;

    -- Example Value: 1/23/20 17:00
""")

┌─────────────────┬──────────────────────────────────┬───────────┬────────┬───────────┬────────┬───────────────┬────────────────┬─────────────────────┬───────────────┬─────────────────────┬───────┬───────┬───────┐
│ len_last_update │                id                │ confirmed │ deaths │ recovered │ active │ incident_rate │ incidence_rate │ case_fatality_ratio │  last_update  │      load_date      │ year  │ month │  day  │
│      int64      │             varchar              │  double   │ double │  double   │ int64  │    double     │     double     │       double        │    varchar    │      timestamp      │ int64 │ int64 │ int64 │
├─────────────────┼──────────────────────────────────┼───────────┼────────┼───────────┼────────┼───────────────┼────────────────┼─────────────────────┼───────────────┼─────────────────────┼───────┼───────┼───────┤
│              13 │ 16f331ef53a23243b8a501196c4dbabc │      NULL │   NULL │      NULL │   NULL │          NULL │           NULL │               

In [27]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT *
    FROM t1
    WHERE len_last_update = 12
    LIMIT 10;

    -- Example Value: 4/6/20 23:22
""")

┌─────────────────┬──────────────────────────────────┬───────────┬────────┬───────────┬────────┬───────────────┬────────────────┬─────────────────────┬──────────────┬─────────────────────┬───────┬───────┬───────┐
│ len_last_update │                id                │ confirmed │ deaths │ recovered │ active │ incident_rate │ incidence_rate │ case_fatality_ratio │ last_update  │      load_date      │ year  │ month │  day  │
│      int64      │             varchar              │  double   │ double │  double   │ int64  │    double     │     double     │       double        │   varchar    │      timestamp      │ int64 │ int64 │ int64 │
├─────────────────┼──────────────────────────────────┼───────────┼────────┼───────────┼────────┼───────────────┼────────────────┼─────────────────────┼──────────────┼─────────────────────┼───────┼───────┼───────┤
│              12 │ 4c7454ac7ae65061c65886d8caf0c9b3 │       6.0 │    0.0 │       0.0 │      6 │          NULL │           NULL │                NUL

In [28]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT *
    FROM t1
    WHERE len_last_update = 11
    LIMIT 10;

    -- Example Value: 3/8/20 5:19
""")

┌─────────────────┬──────────────────────────────────┬───────────┬────────┬───────────┬────────┬───────────────┬────────────────┬─────────────────────┬─────────────┬─────────────────────┬───────┬───────┬───────┐
│ len_last_update │                id                │ confirmed │ deaths │ recovered │ active │ incident_rate │ incidence_rate │ case_fatality_ratio │ last_update │      load_date      │ year  │ month │  day  │
│      int64      │             varchar              │  double   │ double │  double   │ int64  │    double     │     double     │       double        │   varchar   │      timestamp      │ int64 │ int64 │ int64 │
├─────────────────┼──────────────────────────────────┼───────────┼────────┼───────────┼────────┼───────────────┼────────────────┼─────────────────────┼─────────────┼─────────────────────┼───────┼───────┼───────┤
│              11 │ 56d20d41e1cd9cca2769f9a58b381c8d │     990.0 │    6.0 │     984.0 │      0 │          NULL │           NULL │                NULL │ 

## Data Cleansing

Based on the previous queries, we saw that we have different date formats for the last_update field that we need to standardize. This is an important process especially that we are dealing with timeseries dataset.
The following are some of the examples:

- 2023-01-06 04:21:02 => 19 characters
- 2021-01-15 17:22  => 16 characters
- 1/22/2020 17:00 => 15 characters
- 2/1/2020 10:33 => 14 characters
- 1/23/20 17:00 => 13 characters
- 4/6/20 23:22 => 12 characters
- 3/8/20 5:19 => 11 characters

In [37]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT try_strptime(last_update, '%Y-%m-%d %H:%M:%S') AS c_last_update
    FROM t1
    WHERE len_last_update = 19
    LIMIT 20;
""")

┌─────────────────────┐
│    c_last_update    │
│      timestamp      │
├─────────────────────┤
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
│ 2023-01-06 04:21:02 │
├─────────────────────┤
│       20 rows       │
└─────────────────────┘

In [39]:
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT try_strptime(last_update, '%Y-%m-%d %H:%M') AS c_last_update
    FROM t1
    WHERE len_last_update = 16
    LIMIT 20;
""")


┌─────────────────────┐
│    c_last_update    │
│      timestamp      │
├─────────────────────┤
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
│ 2021-01-15 17:22:00 │
├─────────────────────┤
│       20 rows       │
└─────────────────────┘

In [45]:
"""
2/1/2020 10:33 => 14 characters
1/23/20 17:00 => 13 characters
4/6/20 23:22 => 12 characters
3/8/20 5:19 => 11 characters
"""
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT try_strptime(last_update, '%-m/%-d/%Y %H:%M') AS c_last_update
    FROM t1
    WHERE len_last_update = 15
    LIMIT 20;
""")


┌───────────────┐
│ c_last_update │
│   timestamp   │
├───────────────┤
│    0 rows     │
└───────────────┘

In [48]:
"""
2/1/2020 10:33 => 14 characters
1/23/20 17:00 => 13 characters
4/6/20 23:22 => 12 characters
3/8/20 5:19 => 11 characters
"""
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT try_strptime(last_update, '%-m/%-d/%Y %H:%M') AS c_last_update
    FROM t1
    WHERE len_last_update = 14
    LIMIT 20;
""")


┌─────────────────────┐
│    c_last_update    │
│      timestamp      │
├─────────────────────┤
│ 2020-02-01 10:33:00 │
│ 2020-02-01 15:43:00 │
│ 2020-02-01 15:43:00 │
│ 2020-01-31 08:15:00 │
│ 2020-02-01 18:12:00 │
│ 2020-01-31 08:15:00 │
│ 2020-01-31 08:15:00 │
│ 2020-02-01 19:43:00 │
│ 2020-02-01 11:53:00 │
│ 2020-02-01 11:53:00 │
│ 2020-02-01 19:53:00 │
│ 2020-02-01 11:53:00 │
│ 2020-02-01 11:53:00 │
│ 2020-02-01 11:53:00 │
│ 2020-02-01 18:12:00 │
│ 2020-02-01 19:43:00 │
│ 2020-02-01 11:53:00 │
│ 2020-02-01 11:53:00 │
│ 2020-02-01 14:23:00 │
│ 2020-01-31 08:15:00 │
├─────────────────────┤
│       20 rows       │
└─────────────────────┘

In [58]:
"""
1/23/20 17:00 => 13 characters
4/6/20 23:22 => 12 characters
3/8/20 5:19 => 11 characters
"""
con.sql("""
    WITH t1 AS (
        SELECT LENGTH(last_update) AS len_last_update, *
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT try_strptime(last_update, '%-m/%-d/%-y %-H:%-M') AS c_last_update
    FROM t1
    WHERE len_last_update = 13
    LIMIT 20;
""")

┌─────────────────────┐
│    c_last_update    │
│      timestamp      │
├─────────────────────┤
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
│ 2020-01-23 17:00:00 │
├─────────────────────┤
│       20 rows       │
└─────────────────────┘

In [65]:


con.sql("""
    WITH t1 AS (
        SELECT COALESCE(
            try_strptime(last_update, '%Y-%m-%dT%H:%M:%S'),
            try_strptime(last_update, '%Y-%m-%d %H:%M:%S'),
            try_strptime(last_update, '%Y-%m-%d %H:%M'),
            try_strptime(last_update, '%-m/%-d/%Y %H:%M'),
            try_strptime(last_update, '%-m/%-d/%-y %-H:%-M'),
            try_strptime(last_update, '%-m/%-d/%Y %H:%M')
        ) AS c_last_update,
        last_update
        FROM covid19_raw.raw_github_csse_daily
    )
    SELECT *
    FROM t1
""")

┌───────────────┬─────────────┐
│ c_last_update │ last_update │
│   timestamp   │   varchar   │
├───────────────┴─────────────┤
│           0 rows            │
└─────────────────────────────┘

In [90]:
con.sql("SELECT * FROM covid19_raw.raw_github_csse_daily WHERE active > 0 LIMIT 20")

┌──────────────────────────────────┬───────────┬────────┬───────────┬────────┬────────────────────┬────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬───────┬───────┬───────┐
│                id                │ confirmed │ deaths │ recovered │ active │   incident_rate    │ incidence_rate │ case_fatality_ratio │     last_update     │      load_date      │ year  │ month │  day  │
│             varchar              │  double   │ double │  double   │ int64  │       double       │     double     │       double        │       varchar       │      timestamp      │ int64 │ int64 │ int64 │
├──────────────────────────────────┼───────────┼────────┼───────────┼────────┼────────────────────┼────────────────┼─────────────────────┼─────────────────────┼─────────────────────┼───────┼───────┼───────┤
│ d044dd74e9765cfd76636341047342ae │       1.0 │    0.0 │       0.0 │      1 │ 0.9461006461867414 │           NULL │                 0.0 │ 2021-11-01 04:22:01 │ 2021-10-31 

In [73]:
con.sql("""
    SELECT COUNT(CASE WHEN confirmed IS NULL THEN 1 ELSE NULL END) AS confirmed_null,
        COUNT(CASE WHEN deaths IS NULL THEN 1 ELSE NULL END) AS deaths_null,
        COUNT(CASE WHEN recovered IS NULL THEN 1 ELSE NULL END) AS recovered_null,
        COUNT(CASE WHEN active IS NULL THEN 1 ELSE NULL END) AS active_null,
        COUNT(CASE WHEN incident_rate IS NULL THEN 1 ELSE NULL END) AS incident_rate_null,
        COUNT(CASE WHEN case_fatality_ratio IS NULL THEN 1 ELSE NULL END) AS case_fatality_ratio_null
    FROM covid19_raw.raw_github_csse_daily 
""")

┌────────────────┬─────────────┬────────────────┬─────────────┬────────────────────┬──────────────────────────┐
│ confirmed_null │ deaths_null │ recovered_null │ active_null │ incident_rate_null │ case_fatality_ratio_null │
│     int64      │    int64    │     int64      │    int64    │       int64        │          int64           │
├────────────────┼─────────────┼────────────────┼─────────────┼────────────────────┼──────────────────────────┤
│             28 │         433 │        2683864 │     2693277 │             945434 │                   272460 │
└────────────────┴─────────────┴────────────────┴─────────────┴────────────────────┴──────────────────────────┘

In [75]:
con.sql("""
    SELECT MIN(confirmed),
           AVG(confirmed),
           MAX(confirmed)
    FROM covid19_raw.raw_github_csse_daily 
""")

┌────────────────┬───────────────────┬────────────────┐
│ min(confirmed) │  avg(confirmed)   │ max(confirmed) │
│     double     │      double       │     double     │
├────────────────┼───────────────────┼────────────────┤
│      -302844.0 │ 70638.85375641855 │     38487384.0 │
└────────────────┴───────────────────┴────────────────┘

In [76]:
con.sql("""
    SELECT COUNT(1)
    FROM covid19_raw.raw_github_csse_daily 
    WHERE confirmed < 0
""")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│        4 │
└──────────┘

In [77]:
con.sql("""
    SELECT MIN(deaths),
           AVG(deaths),
           MAX(deaths)
    FROM covid19_raw.raw_github_csse_daily 
""")

┌─────────────┬────────────────────┬─────────────┐
│ min(deaths) │    avg(deaths)     │ max(deaths) │
│   double    │       double       │   double    │
├─────────────┼────────────────────┼─────────────┤
│      -178.0 │ 1007.1828728022742 │    185641.0 │
└─────────────┴────────────────────┴─────────────┘

In [78]:
con.sql("""
    SELECT COUNT(1)
    FROM covid19_raw.raw_github_csse_daily 
    WHERE deaths < 0
""")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│        5 │
└──────────┘

In [79]:
con.sql("""
    SELECT MIN(recovered),
           AVG(recovered),
           MAX(recovered)
    FROM covid19_raw.raw_github_csse_daily 
""")

┌────────────────┬────────────────────┬────────────────┐
│ min(recovered) │   avg(recovered)   │ max(recovered) │
│     double     │       double       │     double     │
├────────────────┼────────────────────┼────────────────┤
│      -854405.0 │ 16152.045607604488 │      6399531.0 │
└────────────────┴────────────────────┴────────────────┘

In [80]:
con.sql("""
    SELECT COUNT(1)
    FROM covid19_raw.raw_github_csse_daily 
    WHERE recovered < 0
""")

┌──────────┐
│ count(1) │
│  int64   │
├──────────┤
│        3 │
└──────────┘

In [81]:
con.sql("""
    SELECT MIN(active),
           AVG(active),
           MAX(active)
    FROM covid19_raw.raw_github_csse_daily 
""")

┌─────────────┬────────────────────┬─────────────┐
│ min(active) │    avg(active)     │ max(active) │
│    int64    │       double       │    int64    │
├─────────────┼────────────────────┼─────────────┤
│           0 │ 7469.3125108649165 │     5658278 │
└─────────────┴────────────────────┴─────────────┘

In [86]:
con.sql("""
    SELECT MIN(incident_rate),
           AVG(incident_rate),
           MAX(incident_rate),
           MEDIAN(incident_rate)
    FROM covid19_raw.raw_github_csse_daily 
""")


┌────────────────────┬────────────────────┬────────────────────┬───────────────────────┐
│ min(incident_rate) │ avg(incident_rate) │ max(incident_rate) │ median(incident_rate) │
│       double       │       double       │       double       │        double         │
├────────────────────┼────────────────────┼────────────────────┼───────────────────────┤
│                0.0 │  16507.17332360432 │   2164021.75951979 │    14572.143730514888 │
└────────────────────┴────────────────────┴────────────────────┴───────────────────────┘

In [85]:
con.sql("""
    SELECT MIN(case_fatality_ratio),
           AVG(case_fatality_ratio),
           MAX(case_fatality_ratio),
           MEDIAN(case_fatality_ratio)
    FROM covid19_raw.raw_github_csse_daily 
""")


┌──────────────────────────┬──────────────────────────┬──────────────────────────┬─────────────────────────────┐
│ min(case_fatality_ratio) │ avg(case_fatality_ratio) │ max(case_fatality_ratio) │ median(case_fatality_ratio) │
│          double          │          double          │          double          │           double            │
├──────────────────────────┼──────────────────────────┼──────────────────────────┼─────────────────────────────┤
│                      0.0 │        2.705073356417832 │                  29600.0 │          1.4961915125136018 │
└──────────────────────────┴──────────────────────────┴──────────────────────────┴─────────────────────────────┘

In [92]:
con.sql("SELECT file_md5, load_date FROM delta_scan('s3://covid-data-pipeline/covid19/github_csse_daily') WHERE active > 0 ORDER BY load_date DESC LIMIT 20")


┌──────────────────────────────────┬─────────────────────┐
│             file_md5             │      load_date      │
│             varchar              │      timestamp      │
├──────────────────────────────────┼─────────────────────┤
│ 1de450a5c1a871be31451bd262f3fcb6 │ 2021-10-31 00:00:00 │
│ a297ffde7bf15463a5799b5cdf5266c7 │ 2021-10-30 00:00:00 │
│ c1a3b944c9417545a6f7de1c90d12059 │ 2021-10-29 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00 │
│ 0f54805f0d28b0f391a4ac20d34953bd │ 2021-09-22 00:00:00

In [ ]:


con.sql("SELECT * FROM covid19_raw.raw_github_csse_daily LIMIT 10")

In [132]:
con.sql("""
SELECT id,
       combined_key,
       province_state,
       country_region,
       confirmed * 100000 / coalesce(incident_rate, incidence_rate) AS population,
       confirmed,
       latitude,
       longitude,
       deaths,
       recovered,
       active,
       incident_rate,
       incidence_rate,
       case_fatality_ratio,
       last_update,
       load_date, 
       file_md5
FROM delta_scan('s3://covid-data-pipeline/covid19/github_csse_daily') 
WHERE combined_key = 'Alberta, Canada'
ORDER BY load_date DESC 
LIMIT 50
""")


In [ ]:
con.sql("""
    SELECT gh.load_date,
           cl.location_id,
           cl.admin2,
           TRIM(cl.combined_key, ','),
           deaths AS deaths,
           COALESCE(
            deaths,
            LAST(deaths IGNORE NULLS) OVER (PARTITION BY location_id ORDER BY last_update ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
        ) AS deaths,
    FROM cleansed_github_csse_daily AS gh
    JOIN cleansed_location AS cl ON cl.location_key = gh.id
 WHERE combined_key = 'British Columbia, Canada'
    
--     ORDER BY location_id, last_update 
    LIMIT 10000
""").to_df()

In [6]:
gh_df = con.sql("""
WITH t1 AS (
    SELECT CASE 
                WHEN admin2 IN ('Unassigned', 'Unknown') AND province_state IN ('Unassigned', 'Unknown') THEN 1
                WHEN  admin2 IN ('Unassigned', 'Unknown') AND province_state NOT IN ('Unassigned', 'Unknown') THEN 2
                ELSE 3
           END AS administrative_area_level,
           cl.*,
           gh.*
    FROM cleansed_github_csse_daily AS gh
    JOIN cleansed_location AS cl ON cl.location_key = gh.id
)
SELECT *
FROM t1
WHERE administrative_area_level = 3
""").to_df()

In [22]:
gh_df.shape

(3336784, 22)

In [2]:
import sqlite3
import pandas as pd

In [3]:
sqlite_conn = sqlite3.connect("latest.db")

In [4]:
cursor = sqlite_conn.cursor()

In [7]:
epidemoelogy_df = pd.read_sql("SELECT t.id, t.date AS date_2, administrative_area_level, l.administrative_area_level_1 AS country, l.administrative_area_level_2 AS state, COALESCE(administrative_area_level_3, 'Unassigned') AS city, t.confirmed, t.deaths, t.recovered, t.tests, t.vaccines, t.people_vaccinated, t.people_fully_vaccinated, t.hosp, t.icu, t.vent FROM timeseries t JOIN location l ON l.id = t.id WHERE administrative_area_level = 3 AND date <= '2023-03-09'", con=sqlite_conn)
epidemoelogy_df.to_csv("epidemoelogy.csv", index=False)
del epidemoelogy_df
policies_df1 = pd.read_sql("SELECT t.id, t.date AS date_2, administrative_area_level, l.administrative_area_level_1 AS country, l.administrative_area_level_2 AS state, COALESCE(administrative_area_level_3, 'Unassigned') AS city, t.stringency_index, t.containment_health_index, economic_support_index FROM timeseries t JOIN location l ON l.id = t.id WHERE administrative_area_level = 3 AND date <= '2023-03-09'", con=sqlite_conn)
policies_df1.to_csv("index_policies.csv", index=False)
del policies_df1
policies_df2 = pd.read_sql("SELECT t.id, l.population, school_closing, workplace_closing, cancel_events, gatherings_restrictions, transport_closing, stay_home_restrictions, internal_movement_restrictions, international_movement_restrictions, information_campaigns, testing_policy, contact_tracing, facial_coverings, vaccination_policy, elderly_people_protection FROM timeseries t JOIN location l ON l.id = t.id WHERE administrative_area_level = 3 AND date <= '2023-03-09'", con=sqlite_conn)
policies_df2.to_csv("policy_measures.csv", index=False)
del policies_df2

In [12]:
helper_df.shape

(12054910, 16)

In [ ]:
helper_df.to_csv("covid19_datahub.csv", index=False)

In [78]:
helper_df[["date_2", "country", "state", "city",  "confirmed", "deaths", "recovered"]].tail(10)

,date_2,country,state,city,confirmed,deaths,recovered
780732,2023-02-07,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780733,2023-02-08,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780734,2023-02-09,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780735,2023-02-14,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780736,2023-02-15,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780737,2023-02-16,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780738,2023-02-17,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780739,2023-02-24,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780740,2023-03-03,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN
780741,2023-03-07,Belgium,Ostbelgien,Unassigned,NaN,NaN,NaN


In [19]:
x = pd.merge(
    gh_df,
    helper_df,
    how="left",
    left_on=["load_date", "country_region", "province_state", "admin2"],
    right_on=["date_2", "country", "state", "city"]
)

In [15]:
helper_df[["date_2", "country", "state", "city"]].head(100)

,date_2,country,state,city
0,2020-03-19,Latvia,Vidzeme,Burtnieku novads
1,2020-03-20,Latvia,Vidzeme,Burtnieku novads
2,2020-03-21,Latvia,Vidzeme,Burtnieku novads
3,2020-03-22,Latvia,Vidzeme,Burtnieku novads
4,2020-03-23,Latvia,Vidzeme,Burtnieku novads
5,2020-03-24,Latvia,Vidzeme,Burtnieku novads
6,2020-03-25,Latvia,Vidzeme,Burtnieku novads
7,2020-03-26,Latvia,Vidzeme,Burtnieku novads
8,2020-03-27,Latvia,Vidzeme,Burtnieku novads
9,2020-03-28,Latvia,Vidzeme,Burtnieku novads


In [40]:
gh_df[["load_date", "country_region", "province_state", "admin2", "confirmed", "deaths", "recovered"]].tail(10)

,load_date,country_region,province_state,admin2,confirmed,deaths,recovered
4197101,2020-01-27,Mainland China,Hebei,Unassigned,18.0,1.0,NaN
4197102,2020-01-27,Summer Olympics 2020,Unassigned,Unassigned,0.0,0.0,0.0
4197103,2020-01-27,Malaysia,Johor,Unassigned,0.0,0.0,0.0
4197104,2020-01-27,Malaysia,W.P. Labuan,Unassigned,0.0,0.0,0.0
4197105,2020-01-27,Mainland China,Hubei,Unassigned,1423.0,76.0,45.0
4197106,2020-01-27,Mainland China,Beijing,Unassigned,80.0,1.0,2.0
4197107,2020-01-27,Mainland China,Heilongjiang,Unassigned,21.0,1.0,NaN
4197108,2020-01-27,Mainland China,Gansu,Unassigned,14.0,NaN,NaN
4197109,2020-01-27,Malaysia,Terengganu,Unassigned,0.0,0.0,0.0
4197110,2020-01-27,Antarctica,Unassigned,Unassigned,0.0,0.0,0.0


In [20]:
x[["load_date", "country_region", "province_state", "admin2", "confirmed_x", "deaths_x", "recovered_x", "date_2", "country", "state", "city", "confirmed_y", "deaths_y", "recovered_y", "vaccines"]].head(100)

,load_date,country_region,province_state,admin2,confirmed_x,deaths_x,recovered_x,date_2,country,state,city,confirmed_y,deaths_y,recovered_y,vaccines
0,2023-01-12,Malaysia,Pulau Pinang,Unassigned,307228.0,2064.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-12,New Zealand,Niue,Unassigned,699.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-12,US,Connecticut,Unassigned,3192.0,11.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-12,US,Georgia,Out of GA,97487.0,1610.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-12,US,Illinois,Out of IL,3.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2023-01-12,US,Louisiana,Out of LA,0.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023-01-12,US,North Dakota,Unassigned,0.0,196.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2023-01-12,US,Puerto Rico,Out of PR,434.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2023-01-12,US,Rhode Island,Unassigned,29887.0,2.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2023-01-12,US,Virginia,Unassigned,0.0,0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
